In [ ]:
pip install geopy


In [ ]:
import pandas as pd 
import numpy as np 
import math 
import os
import datetime
from datetime import date, datetime, timedelta
import re
import ast
import geopy

## 1 Checking dimensions tables.

In [ ]:
dimension_table = pd.read_excel('Inputs/User Details.xlsx', sheet_name = None,  parse_dates=True )

In [ ]:
if 'pandas' in str(type(dimension_table)): 
    print(dimension_table)
elif 'dict' in str(type(dimension_table)):
    for dimension in dimension_table:
        print('---------------------------------------------------------------------------------------')
        print(dimension)
        print('---------------------------------------------------------------------------------------')
        print(dimension_table[dimension])
        print('---------------------------------------------------------------------------------------')

In [ ]:
accounts = dimension_table['Accounts']
assets = dimension_table ['Assets']



In [ ]:
print(accounts.info())
print('--------------------------------------------')
print(accounts.duplicated(subset='AccountID').sum())

<b>So, there are duplicates</b> in primary key for Accounts table. Not good. Should be reported to developers. 
Can be resolved in many ways: 
- 1.Try to find out which AccountID should actually have each of accounts in alternative source of truth (if there is any), report to devs and to fix it for now as ad hoc. 
- 2. Drop duplicate (second one by default). 
- 3. To set it last existing id in current table +1 at least for now in case I need to calculate some stats in account table. 

Let's have a look on, probably, connected assets table though. 

Datatypes looks right. 


In [ ]:
assets.shape[0]

In [ ]:
print(assets.info())
print()
print(assets.nunique())

In [ ]:
def deduplicator(df, primary_keys): 
    if df.duplicated(subset = primary_keys).sum() > 0: 
        df.drop_duplicates(subset = primary_keys, inplace = True, ignore_index= True)
        return None #todo: write amount of deduplicated rows in a list -part of dict, which will be then dict of metadata. 

In [ ]:
deduplicator(accounts, 'AccountID')

In [ ]:
deduplicator(assets, 'AssetID')

In [ ]:
def values_mapper(df, column_name, mapper):
    new_col = column_name + '_fixed'
    df[new_col] = df[column_name].map(mapper)
    df[new_col] = df.apply(lambda x: x[column_name] if pd.isna(x[new_col]) else x[new_col], axis =1 )

In [ ]:
assets_fix = {'Renaut': 'Renault'}
values_mapper(assets, 'Make', assets_fix)
print(assets)
              

In [ ]:
assets.rename(columns = {'Make': "maker", 'Make_fixed': 'maker_fixed', 'AssetID': 'asset_id', 
                         'License Plate': 'license_plate', 'Model': 'model', 'AccountID':'account_id', 
                         'Datasource': 'data_source', 'Tank Capacity': 'tank_capacity'
                        }, inplace = True)

In [ ]:
accounts.rename(columns= {'AccountID': 'account_id', 'AccountName':'account_name'}, inplace = True)

In [ ]:
accounts

In [ ]:
print(accounts.info())
print()
print('----------------')
print()
print(assets.info())

Talbes cleared. Let.s leave it


## Facts tables cleaning

In [ ]:
def fact_files_to_read(input_directory):
    objects = os.scandir(inputs_dir)
    fileList = []
    for obj in objects:
        if obj.is_file() and re.fullmatch('\d*\.xlsx', obj.name) is not None:
            fileList.append(inputs_dir+obj.name)
    return fileList 

In [ ]:
inputs_dir= 'Inputs/'
files = fact_files_to_read(inputs_dir)
files

In [ ]:
def facts_reader(files): 
    dfs= []
    for file in files: 
        df=pd.read_excel(file, parse_dates=[1])
        dfs.append(df)
    return dfs 
        

In [ ]:
fact_tables = facts_reader(files)

In [ ]:
for fact_table in fact_tables:
    print(fact_table.columns)
    print('-------')
    print()   
#Typos in column names (whitespaces), also different names for some reason. And Odometer Mts and without Mts, Pos (lat, lon)
#and without (Lat, Long) - different formats? 
#Timestamp with format only in first file. Need to check the data and it's datatypes then. 

In [ ]:
#Visually check data and data types. 
for fact_table in fact_tables:
    print(fact_table.head())
    print('-------------------')
    print()
    print('-------------------')
# See different column names. Needs to be fixed. 
#also - unexpected nulls in ignition for 334455, 334458 , odometer nulls, fueel %age. 

In [ ]:
fact_tables[0].info() #some of pos are nulls, most of ignition, fueld, odometer values are nan. Need to fix or drop nulls in the future.
#assumption: pair of ID+timestamp = natural key. 

In [ ]:
print(fact_tables[0][' Ignition'].dropna().values) #integers, but not so many. 
print('-------------')
print()
print('-------------')
print(fact_tables[0][' Odometer'].dropna().values)  # so,integers in reality, floats in table with  in the middle for some reason. 
print('-------------')
print()
print('-------------')
print(fact_tables[0][' Fuel %age'].dropna().values) #integers in reality, float in table, but not so many. 

In [ ]:
fact_tables[0].info() #at least no nulls. 

In [ ]:
print(fact_tables[1][' Ignition'].dropna().values) #ingegers
print('-------------')
print()
print('-------------')
print(fact_tables[1][' Odometer Mts'].dropna().values)  # floats 
print('-------------')
print()
print('-------------')
print(fact_tables[1][' Fuel %age'].dropna().values) #integers


#also observed all other files in the same way and...

In [ ]:
#... and now I can at least rename columns for all files and to convert datatypes to the most suitable
#wide type: if the data in one file in column_x is int and in another file is float - it'll be float, as a type, preserving 
#accuracy of both floats and ints. 
def columns_common_names(list_df, cols_list):
    for df in list_df: 
        df.columns = cols_list
    return None
# To convert cols to common datatypes, but after I can get rid of nans. 
def columns_common_datatypes(list_df, cols_convert_map):
    for df in list_df: 
        for col in cols_convert_map:
            df[col]=df[col].apply(cols_convert_map[col])
    return None

In [ ]:
facts_cals = ['id', 'timestamp', 'ignition', 'odometer_m', 'pos', 'fuel_%']
convert_map = {
    'ignition': int, 
    'odometer_m': float, 
    'fuel_%': int    
}

columns_common_names(fact_tables, facts_cals)
#columns_common_datatypes(fact_tables, convert_map)

In [ ]:
for fact in fact_tables: 
    print(sum(fact.duplicated(subset=['id', 'timestamp']))) 
#quite unexpected. Means - id+timestamp isn't unique and 
#we actually can recieve >1 events in the same second. Can be resolved either by bigger granularity scale (milliseconds)
#or to add events_sequence. Let's check if id+timestamp+ignition are unique. 

In [ ]:
for fact in fact_tables: 
    print(sum(fact.duplicated(subset=['id', 'timestamp', 'ignition'])))
#yes, at least it's totally unique. But as there are null values for ignition in some tables, makes sense to fill in the values
#based either on: previous(initial) state. Or: initial state + movement fact (to check delta of distance between rows, if 
#distance covered >0, then ignition =1, if =0 - then...well, then it can be either ignition = 0 as well as ignition = 1 (you still
#can leave the ignited car or stay in traffic jam.)). 

#I think we can believe ignition sensors work, but work in different way, so in some cases they're producing results constantly,
#and sometimes - ony during state changes. 

In [ ]:
first_sample = fact_tables[0]
first_sample['is_ignition_value']=first_sample['ignition'].apply(lambda x: 0 if pd.isna(x) else 1)#could be fill na straightforward
#but decided to implement through table functions

In [ ]:
#first_sample['ignition_parts']= first_sample.sort_values('timestamp',  ascending=True).groupby(['id'])['ignition'].rank()



first_sample['ignition_parts']=first_sample.sort_values('timestamp')['is_ignition_value'].cumsum()
first_sample['ignition_filled'] = first_sample.sort_values('timestamp').groupby(['id','ignition_parts'])['ignition'].ffill()
first_sample['ignition_filled'] = first_sample['ignition_filled'].apply(int)


first_sample['prev_ignition'] = first_sample.sort_values('timestamp').groupby(['id'])\
['ignition_filled'].shift(1)
first_sample['changed_ignition_status'] = first_sample['ignition_filled']!=first_sample['prev_ignition']
first_sample['seq_number']=first_sample.sort_values(['timestamp', 'changed_ignition_status']).groupby(['id', 'timestamp']).cumcount()+1

first_sample=first_sample[['id', 'timestamp', 'seq_number', 'ignition', 'ignition_filled', 'odometer_m', 'pos', 'fuel_%']]

In [ ]:
def order_and_ignition_fill (df): 
    df['is_ignition_value']=df['ignition'].apply(lambda x: 0 if pd.isna(x) else 1)
    df['ignition_parts']= df.sort_values('timestamp')['is_ignition_value'].cumsum()
    df['ignition_filled'] = df.sort_values('timestamp').groupby(['id','ignition_parts'])['ignition'].ffill()
    df['ignition_filled'] = df['ignition_filled'].apply(int)
    
    #Sequence for the events, so engine_off should always be after engine on sent in the same time. 
    df['prev_ignition'] = df.sort_values('timestamp').groupby(['id'])['ignition_filled'].shift(1)
    df['changed_ignition_status'] = df['ignition_filled']!=df['prev_ignition']
    df['seq_number']=df.sort_values(['timestamp', 'changed_ignition_status']).groupby(['id', 'timestamp']).cumcount()+1
    return None





In [ ]:
def ignition_fill_all_tables(dflist):
    for df in dflist: 
        order_and_ignition_fill(df)
    return None

ignition_fill_all_tables(fact_tables)
for i, df in enumerate(fact_tables): 
    fact_tables[i]=fact_tables[i][['id', 'timestamp', 'seq_number', 'ignition', 'ignition_filled', 'odometer_m', 'pos', 'fuel_%']]


In [ ]:
##Also it's possible to fill in coordinates and fuel, as well as odometer metric (should be nearly the same in the same moment of time) for those stop engine events, which happened the same second we have data for 
##start engine:
def fill_in_pos_fuel_od_ign_off(df):
    df['pos']=df.sort_values(['id','timestamp', 'seq_number']).groupby(['id', 'timestamp'])['pos'].ffill()
    df['fuel_%']=df.sort_values(['id','timestamp', 'seq_number']).groupby(['id', 'timestamp'])['fuel_%'].ffill()
    df['odometer_m']=df.sort_values(['id','timestamp', 'seq_number']).groupby(['id', 'timestamp'])['odometer_m'].ffill()
    return None
    
def fill_same_second_fuel_pos_gaps(dflist):
    for df in dflist:
        fill_in_pos_fuel_ign_off(df)
    return None

In [ ]:
fill_same_second_fuel_pos_gaps(fact_tables)

In [ ]:
for df in fact_tables:
    print(df.info())
    print()
    print('-------------------------------------')
    print()

In [ ]:
#Okay, so ignition status is recovered. All the events have geo, but not all have odometer values. So, it makes sense to recover
#distance covered from geo. Will be alternative source of truth for o distance covered metric, as well, as a control metric for
#odometer. 

#And....let's put  latitude and longitude in separate columns too!


In [ ]:
#fact_tables[0]['prev_pos']=fact_tables[0].sort_values(['id', 'timestamp', 'seq_number']).groupby('id')['pos'].shift()

In [ ]:
def geo_extractor (string_tuple):
    """Allows to extract latitude and longitude from 
    stringified tuples of pos column. 
    """
    lat = np.nan
    lon = np.nan
    try: 
        lat = ast.literal_eval(string_tuple)[0]
        lon = ast.literal_eval(string_tuple)[1]
    except ValueError:
        print('Input is not a tuple')
    return lat, lon
    


def prev_pos (df): 
    """Creates geo-cols in dataframe"""
    df['prev_pos'] = df.sort_values(['id', 'timestamp', 'seq_number']).groupby('id')['pos'].shift()
    df['current_lat'] = df['pos'].apply(lambda x: geo_extractor(x)[0])
    df['current_lon'] = df['pos'].apply(lambda x: geo_extractor(x)[1])
    df['prev_lat'] = df['prev_pos'].apply(lambda x: geo_extractor(x)[0])
    df['prev_lon'] = df['prev_pos'].apply(lambda x: geo_extractor(x)[1])
    #df['current_lon'] = df['pos'].apply(lambda x: ast.literal_eval(x)[1])
    #df['prev_lat'] = df['prev_pos'].apply(lambda x: ast.literal_eval(x)[0])
    #df['prev_lon'] = df['[prev_pos'].apply(lambda x: ast.literal_eval(x)[1])
    return None
    
def coordinates_parser(df_list): 
    """Iterates through the list of DFs to apply functions"""
    for df in df_list: 
        prev_pos(df)
    return None


In [ ]:
coordinates_parser(fact_tables)

In [ ]:
    ex4_pandas.sort_values("Date")
    .groupby("ticker")["closing_price"]

In [ ]:
print(fact_tables[3].head(50))

In [ ]:
fact_tables[1].to_csv('second_sample.csv')
fact_tables[2].to_csv('third_sample.csv')
fact_tables[3].to_csv('fourth_sample.csv')
fact_tables[4].to_csv('fifth_sample.csv')


In [ ]:
fact_tables[0].to_csv('first_sample.csv')

In [ ]:
for fact in fact_tables: 
    order_and_ignition_fill(fact)

In [ ]:
fact_tables[0]

In [ ]:
first_sample.duplicated(subset = ['id', 'timestamp', 'seq_number'])

In [ ]:
first_sample.to_csv('first_sample.csv')

In [ ]:
first_sample

In [ ]:
first_sample.columns = facts_cals
first_sample


In [ ]:
first_sample.columns

In [ ]:
fifth_sample

In [ ]:
fourth_sample.columns

In [ ]:
dataframes = []

s=re.fullmatch('\d*\.xlsx', '334455.xlsx')
if s is not None: 
    print(s.string)
    pd.pandas.read_excel(, sheet_name=0, *, 
                         header=0, names=None, index_col=None, usecols=None, 
                         dtype=None, engine=None, converters=None, true_values=None, 
                         false_values=None, skiprows=None, nrows=None, na_values=None, 
                         keep_default_na=True, na_filter=True, verbose=False, 
                         parse_dates=False, date_parser=_NoDefault.no_default, 
                         date_format=None, thousands=None, decimal='.', comment=None, 
                         skipfooter=0, storage_options=None, dtype_backend=_NoDefault.no_default, engine_kwargs=None)